In [ ]:
#merge all article files(excel) in one dataframe
import pandas as pd
import glob
#making list of strings from txt files in articles folder
path = './Articlespk/*xlsx' # path of files
files = glob.glob(path)

all_articles = pd.DataFrame() #all articles with article id
for f in files:
    df = pd.read_excel(f)
    print(len(df))
    all_articles = all_articles.append(df,ignore_index=True, sort=True)


In [ ]:
#data extraction from xml(the data provided by client for article content)
import lib
import xml.etree.ElementTree as ET
import pandas as pd
import re

#making list of dictionaries where 1 dictionary= 1 article data
def remove_tags(text):
    TAG_RE = re.compile(r'<[^>]+>')
    temp= TAG_RE.sub('', text)
    temp= temp.replace('\n', " ")
    temp=temp.replace('&nbsp;',' ')
    return temp

def xml_to_excel(fileurl,excel_name):
    tree = ET.parse(fileurl)
    root = tree.getroot()
    list_of_dict=[]
    for article in root:
        temp={}
        temp[article[0].tag] = article[0].text
        temp[article[1].tag] = article[1].text
        temp[article[2].tag] = article[2].text
        temp[article[4].tag] = article[4].text
        temp[article[5].tag] = article[5].text
        temp[article[6].tag] = article[6].text
        temp[article[7].tag] = article[7].text
        #temp[article[9].tag] = article[9].text

        list_of_dict.append(temp)

    #may_dict = xml_to_dict("May2019.xml")
    dic_df = pd.DataFrame.from_dict(list_of_dict, orient='columns')
    dic_df["dtls"] = pd.Series([remove_tags(x) for x in dic_df["dtls"]])
    dic_df.to_excel(excel_name+'.xlsx')
    trans = dic_df[['dtls', 'news_id']]
    trans.to_excel(excel_name+'_trans.xlsx')





In [ ]:
#filtering rows and extracting article id example:(it mayt be different depending on your data)
#filtering unnecessary rows example
GA_list=[]#weekday is data extracted from GA
for row in weekday.itertuples(index=False):
   if re.findall("-\d*\d\d\d\d\d", row.Page):
       GA_list.append(row)
weekday_clean=pd.DataFrame(GA_list)
ids=[]
#extracting ids
for row in weekday_clean.itertuples(index=False):
    try:
        temp=re.findall("-\d*\d\d\d\d\d", row.Page)[0].split("-", 1)[-1]
        ids.append(temp)
    except:
        print(row.Page)
ids=pd.Series(ids)
weekday_clean["news_id"]=ids
del autho_clean["Page"]
autho_clean["news_id"]=autho_clean["news_id"].astype("int")
autho_clean=autho_clean.drop_duplicates()
autho_clean=autho_clean.sort_values(["news_id"])

In [ ]:
import datetime
from datetime import timedelta 
dts=[datetime.datetime.strptime(str(x),"%Y%m%d%H")  for x in GA_clean["hourofday"] ]
GA_clean["hourofday"]=pd.Series(dts)
GA_xml=pd.merge(GA_clean,all_data[["crtd_date", "news_id"]], on= "news_id")
#crtd date
crtd=[datetime.datetime.strptime(x.rsplit("+", 1)[0].rsplit('.',1)[0], '%Y-%m-%dT%H:%M:%S') for x in GA_xml["crtd_date"]]
GA_xml["crtd_date"]=pd.Series(crtd)
limit3hr= []
for row in GA_xml.itertuples():
    if row.crtd_date.minute > 30:
        temp= row.crtd_date + timedelta(hours=3)
    else:
        temp= row.crtd_date + timedelta(hours=2)
    limit3hr.append(temp)

GA_xml["limit"]= pd.Series(limit3hr)





#summing up views for first 3 hours
GA3= [] #only relevant rows for next 3 hr views
for row in GA_xml.itertuples():
    if row.hourofday <= row.limit:
        GA3.append(row)

GA3=pd.DataFrame(GA3)


GA3final=GA3.groupby(["news_id", "crtd_date", "limit"], as_index=False). agg({"Pageviews":sum, "uniquepv":sum})
#GA3final["news_id"]= pd.Series([re.split("/",x)[0] for x in GA3final[""]])

#GA3_final has only articles with nonzero views in first three hours. we want all articles with 0 as views count in first 3 hours if no views.
GA_xml_grouped=GA_xml.groupby(["news_id", "crtd_date", "limit"], as_index=False).count()
GA_xml_grouped.drop(columns=['Page', 'hourofday', 'Pageviews','uniquepv'], axis=1, inplace=True)

GA_final_withallids= pd.merge(GA_xml_grouped,GA3final, on= ['news_id', 'crtd_date', 'limit'], how= "left")
GA_final_withallids.fillna(0, inplace=True)#final data for first 3 hour views for all 87431 articles

GA3_final_withallids=pd.read_pickle("GA3_final_withallids")#first 3 hr views
GA_final_withallids.to_pickle("GA3_final_withallids")
#summing up first 3 days views
#pass hourwise dataframe in data here GA_xml-it must have columns Index(['news_id', 'crtd_date', 'Page', 'hourofday', 'Pageviews',
      # 'uniquepv', 'month'],
def add_ndays_views(data, day):
    #limit= []
    #for row in data.itertuples():
       # temp= row.crtd_date + timedelta(days =3)
        #limit.append(temp)

    #data["limit"]= pd.Series(limit)
    #del limit
    data["limit"] = data.apply(lambda row: row["crtd_date"] + timedelta(days=3), axis=1)
    # summing up views for first ndays
    relevant = []  # only relevant rows for next n days views
    for row in data.itertuples():
        if row.hourofday <= row.limit:
            relevant.append(row)

    relevant_data = pd.DataFrame(relevant)
    del relevant
    relevant_data_grouped = relevant_data.groupby(["news_id", "crtd_date", "limit"], as_index=False).agg({"Pageviews": sum, "uniquepv": sum})
    #zeroviews = set(data["news_id"]) - set(relevant_data["news_id"])
    #data.loc[data["news_id"]== 948234].index
    #data.loc[1037195, :]checking
    data_grouped = data.groupby(["news_id", "crtd_date", "limit"], as_index=False).count()
    data_grouped.drop(columns=['Page', 'hourofday', 'Pageviews', 'uniquepv'], axis=1, inplace=True)

    final_summed_views = pd.merge(data_grouped, relevant_data_grouped, on=['news_id', 'crtd_date', 'limit'], how="left")
    final_summed_views.fillna(0, inplace=True)  # final data for first n days views for all 87431 articles

    return  final_summed_views


In [ ]:
#extracting number of entities and other entities from content:
i1=["PERSON", "NORP", "ORG", "GPE"]#included in number of entities
i2=[ "FAC", "LOC", "PRODUCT", "EVENT", "WORK_OF_ART"]#included in other  entities
import spacy
import pandas as pd
nlp = spacy.load('en', disable=["parser"])
path = '/home/dhruvpatel/PycharmProjects/Virality/Articlespk/*.xlsx'
files = glob.glob(path)

all_articles = pd.DataFrame() #all articles with article id
for f in files:
    df = pd.read_excel(f)
    print(len(df))
    all_articles = all_articles.append(df,ignore_index=True, sort=True)
df_list=list(all_articles["english"])
# included=[ "PERSON", "NORP", "FAC", "ORG", "GPE", "LOC", "PRODUCT", "EVENT", "WORK_OF_ART"]
# #"WORK_OF_ART", "LAW", "LANGUAGE", "DATE", "TIME", "PERCENT" "MONEY", "QUANTITY", "ORDINAL", "CARDINAL",
# list_of_entities= []
# for i in df_list:
#     temp=nlp(i)
#     li=[x for x in temp.ents if x.label_  in included]
#     list_of_entities.append(li)

i1=["PERSON", "NORP", "ORG", "GPE"]
i2=[ "FAC", "LOC", "PRODUCT", "EVENT", "WORK_OF_ART"]
num_of_entities=[]
other_entities=[]
for i in df_list.itertuples(index=False):
    temp=nlp(i)
    e1=0
    oe=0
    for x in temp.ents:
        if x.label_ in i1:
            e1+=1
        if x.label_ in i2:
            oe+=1
    num_of_entities.append(e1)
    other_entities.append(oe)
all_articles["num_of_entities"]=pd.Series("num_of_entities")
all_articles["other_entities"]=pd.Series("other_entities")

In [ ]:
#getting article category
GA_xml= pd.read_pickle("GA_xml.pickle")
GA_xml=GA_xml.reset_index()
cat=[]
for i in range(len(GA_xml)):
    try:
        temp=re.findall("/[a-zA-Z0-9-]*/news/", GA_xml.Page[i])[0].split("/")[1]
        cat.append(temp)
    except:
        #print(row.Page)
        temp="na"
        cat.append(temp)
len(cat)
c=0
for i in cat:
    if i=="na":
        c+=1
GA_xml["category"]=pd.Series(cat)
GA_xml2=GA_xml.loc[GA_xml["category"]!="na", :]
GA_xml3=GA_xml2.loc[:,["news_id", "category"]]
GA_xml3.drop_duplicates(inplace=True)
tr=GA_xml3.drop_duplicates(subset='news_id', keep="last")
p=tr["category"].value_counts()
#a57 diferent categories. we will group them in 5 categories.
cate=[]
for i in tr["category"]:
    if i in ["national"]:
        cate.append("national")
    elif i in ["himachal-pradesh", "uttar-pradesh","punjab","haryana", "madhya-pradesh","jalandhar","chandigarh", "uttrakhand", "ludhiana", "amritsar", "jharkhand", "gurdaspur", "firozepur" ,"hoshiarpur","bathinda"
               "patiala","barnala","faridkot","kanpurthala", "nawanshahr","ambala", "faridabad", "panipat","khanna","yamunanagar","sonipat", "gurgaon","kurukshetra","karnal", "jind", "sirsa" , "bhiwani"
               "sirsa", "bhiwani", "kaithal", "rphtak","fatehabad","new-delhi","jammu-kashmir"]:
        cate.append(("regional"))
    elif i in ["sports","cricket"]:
        cate.append("sports")
    elif i in ["entertainment"]:
        cate.append("entertainment")
    elif i in ["business"]:
        cate.append("business")
    elif i in ["international"]:
        cate.append("international")
    elif i in ["education-and-jobs"]:
        cate.append("education-and-jobs")
    elif i in ["nari"]:
        cate.append("nari")
    elif i in ["dharm"]:
        cate.append("dharm")
    elif i in ["life-style"]:
        cate.append("life-style")
    elif i in ["blogs"]:
        cate.append("blogs")
    else:
        cate.append("other")

#final data
tr.reset_index(inplace=True)
tr["cat"]=pd.Series(cate)
tr["cat"].value_counts()
del tr["level_0"]
del tr["index"]
del tr["category"]
tr.to_pickle("category_id.pickle")
fi=pd.read_pickle("feed_data5.pickle")
#fi=fi.drop(columns=["english"])
final=pd.merge(fi,tr, on="news_id")
final["weekday"]=final["crtd_date"].apply(lambda x: x.weekday())# Monday is 0 and Sunday is 6
final["hour"]=final["crtd_date"].apply(lambda x: x.hour)
final.to_pickle("feed_data5.pickle")

In [ ]:
#translation
from googletrans import Translator
def hi_to_eng(text):
    translator = Translator()
    temp=translator.translate(text,dest="en")
    return temp.text


In [ ]:
#text stats features making
from textblob import TextBlob
text = "this movie was piece of shit."

blob = TextBlob(text)

import pandas as pd
import glob
#making list of strings from txt files in articles folder
path = '/home/dhruvpatel/PycharmProjects/Virality/Articlespk/*.xlsx'
files = glob.glob(path)

all_articles = pd.DataFrame() #all articles with article id
for f in files:
    df = pd.read_excel(f)
    print(len(df))
    all_articles = all_articles.append(df,ignore_index=True, sort=True)


polarity=[]
subjectivity=[]
for row in all_articles.itertuples(index=True):
    blob=TextBlob(row.english)
    polarity.append(blob.sentiment.polarity)
    subjectivity.append(blob.sentiment.subjectivity)

all_articles["polarity"]=pd.Series(polarity)
all_articles["subjectivity"]=pd.Series(subjectivity)
num_of_sentences=[]
num_of_syllables=[]
length=[]
from textstat.textstat import textstat
for row in all_articles.itertuples(index=True):
    num_of_sentences.append(textstat.sentence_count(row.english))
    #num_of_syllables.append(textstat.syllable_count(row.english))
    length.append(len(row.english))

all_articles["num_of_sentences"]=pd.Series(num_of_sentences)
#all_articles["num_of_syllables"]=pd.Series(num_of_syllables)
all_articles["length"]=pd.Series(length)
